In [ ]:
import time
from multiping import multi_ping
from flask import request, Response
#from flask import Flask
from config import netbox_api
from jinja2 import Environment, FileSystemLoader
from nornir_napalm.plugins.tasks import napalm_get, napalm_configure
import ipaddress
from nornir_utils.plugins.functions import print_result
import json
from nornir.core.task import Task, Result
from nornir_jinja2.plugins.tasks import template_file
from nornir_netmiko.tasks import netmiko_send_command, netmiko_send_config

import re

from credentials import(netbox_url,
                        netbox_token,
                        device_username,
                        device_password)
from nornir import InitNornir
from nornir.core.filter import F
from deepdiff import DeepDiff
from deepdiff import grep

In [ ]:
def create_nornir_session():
    """ 
    Инициализируем nornir, но для "hosts" используем данные из netbox
    :return: nr_session
    """
    nr_session = InitNornir(
        inventory={
            "plugin": "NetBoxInventory2",
            "options": {
                "nb_url": netbox_url,
                "nb_token": netbox_token,
                "group_file": "./inventory/groups.yml",
                "defaults_file": "./inventory/defaults.yml",
            },
        },
    )
    return nr_session

In [ ]:
templates_path = "./templates/"

Заполняем журнал

In [ ]:
def journal_template_fill(comment,level):
    """ 
    Заполнение шаблона значениями
    :param comment: событие 
    :param level: уровень важности
    :return: journal
    """  
    template_file = "netbox_journals.template"
    environment = Environment(loader=FileSystemLoader(templates_path)) # загружаем шаблон для заполнения
    template = environment.get_template(template_file)
    journal = None
    wow = ''
    netbox_level = ''
    
    kind = ['emergency','alert','critical','error','warning','notification','informational','debugging',]
    set_kind = tuple(kind) # преобразуем список в кортеж
    
    if level in kind: 
        log_level = set_kind.index(level)
        
        if log_level in range(2):
            netbox_level = 'danger'
            wow = '!!!'
        
        elif log_level in range(2,5):
            netbox_level = 'warning'
            wow = '!'
        
        elif log_level in range(5,7):
            netbox_level = 'info'
            wow = '.'
    
    else: 
        netbox_level = 'success'
        wow = ''
    
    try:
        journal = template.render( # заполняем шаблон
                                assigned_object_type = 'dcim.device',
                                assigned_object_id = '12',
                                kind = netbox_level,
                                comments = comment + '{}'.format(wow)
                                )
    except: print("not all arguments have been transmitted...")
    else: 
    
        if journal != None:
            journal = json.loads(journal)
            netbox_api.extras.journal_entries.create([journal])
            print(level.upper()+': '+comment+wow)

Создаем функцию проверки None

In [ ]:
def conversion(tup, dict = {}):
    for x, y in tup:
        dict.setdefault(x, []).append(y)
    return dict

In [ ]:
"""
Common (generic) functions that can be imported by any script/module.
"""
import re


def parse_interface_name(interface_name):
    """
    Given an interface name, split the string into the type of interface
    and the interface ID.  Use for generating RESTCONF URLs requiring an
    interface specifier.
    :param interface_name: String - name of the interface to parse
    :return: Tuple of (interface type, interface ID)
    """
    interface_pattern = r"^(\D+)(\d+.*)$"
    interface_regex = re.compile(interface_pattern)

    interface_type, interface_id = interface_regex.match(str(interface_name)).groups()

    return interface_type, interface_id

Создаем функцию извлечения IPv4

In [ ]:
def configure_interface_ipv4_address(netbox_ip_address):
    """
    Извлечение IPv4 адреса, маски, сети, префикса, шлюза.
    :param netbox_ip_address: IPv4 адрес (IP/Prefix)
    :return: ip4_address
    """
    ipv4_dic = dict()
    ipv4_dic['ip4_address'] = format(ipaddress.IPv4Interface(netbox_ip_address).ip)
    ipv4_dic['ip4_netmask'] = format(ipaddress.IPv4Interface(netbox_ip_address).netmask)
    ipv4_dic['ip4_network'] = format(ipaddress.IPv4Interface(netbox_ip_address).network)
    ipv4_dic['ip4_prefix'] = format(ipaddress.IPv4Network(ipv4_dic['ip4_network']).prefixlen)
    ipv4_dic['ip4_broadcast'] = format(ipaddress.IPv4Network(ipv4_dic['ip4_network']).broadcast_address)
    ipv4_dic['ip4_gateway'] = format(list(ipaddress.IPv4Network(ipv4_dic['ip4_network']).hosts())[-1])

    return(ipv4_dic)
#configure_interface_ipv4_address('10.30.1.105/24')['ip4_address']

Создаем функцию получения адреса management интерфейса

In [ ]:
def mgmt_address(device_interface):
    """
    Извлечение IPv4 адреса интерефейса управления.
    :param device_interface: ссылка на объект интерфейса pynetbox
    :return: ip4_address
    """    
    interface_type, interface_id = parse_interface_name(device_interface.name)
    mgmt_ip = configure_interface_ipv4_address(device_interface.device.primary_ip)['ip4_address']
    
    return(mgmt_ip)

In [ ]:
def compare(prechange, postchange):
    compare = DeepDiff(prechange,postchange,exclude_paths="root['last_updated']")
    change = dict()
    change_key = []
    new_value = []

    for key in compare.keys():
        
        if key == 'values_changed' or key == 'type_changes':
            
            for inkey in compare[key].keys():
                change_key.append(re.findall("'([^']*)'", inkey)[0])
                new_value.append(compare[key][inkey]['new_value'])

    change = dict(zip(change_key,new_value))
    
    if len(change) == 0:
        change = None
    
    #print('Configuration changes:\n\t{}'.format(change))
    
    return change

In [ ]:
def convert_none_to_str(value):
    return '' if value is None else str(value)

Заполняем шаблон

In [ ]:
def cisco_config_interface(j2_interface,event='None'):
    """ 
    Заполнение шаблона значениями, если событие "delete", то заполняем по умолчанию
    :param j2_interface: ссылка на объект интерфейса pynetbox 
    :param event: событие
    :return: None
    """   

    def template_fill(*args,**kwargs):
        environment = Environment(loader=FileSystemLoader(templates_path)) # загружаем шаблон для заполнения
        template = environment.get_template(template_file)
        content = None
        try:
            if event == 'shutdown':
                content = template.render( # заполняем шаблон
                                    interface_name = convert_none_to_str(j2_interface.name)
                                )
            else:
                content = template.render( # заполняем шаблон
                                    interface_name = convert_none_to_str(j2_interface.name),
                                    descr = convert_none_to_str(j2_interface.description),
                                    access_vlan = convert_none_to_str(j2_interface.untagged_vlan.vid),
                                    mode = convert_none_to_str(j2_interface.mode.value)
                                )
        except: 
            # > добавляем запись в журнал
            comment,level =  'Not enough data to fill out the template','warning'                
            journal_template_fill(comment,level)
            # <
        else:
            #print("Filling in the template...\n{}".format(content)) 
            print("Filling in the template...")

        return content 
        
    if event == 'shutdown':
        template_file = "cisco_ios_shutdown_interface.template"
        content = template_fill(j2_interface, template_file,event)
    
    elif event != 'delete':    
        template_file = "cisco_ios_access_interface.template"
        content = template_fill(j2_interface, template_file)
    
    else:
        template_file = "cisco_ios_default_interface.template"
        content = template_fill(j2_interface, template_file)

    return content

#get_device_interface = netbox_api.dcim.interfaces.get(136)
#cisco_config_interface(get_device_interface, event='update')

Получаем через napalm интерфейсы с устройства

In [ ]:
def push_config_interface(netbox_interface,content,event='None'):
    """  
    Проверка на доступность устройства (ping)
    Подключение к устройству по IP адресу, принадлежащему интерфейсу управления
    Проверка соответствия портов между netbox и реальным устройством
    Отправка конфигурации на устройство
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :return: None
    """
    attempts = 3 # количество попыток подключения
    attempt_timeout = 5 # время ожидания между попытками в секундах
    fail_count = 0 # количество неудачных попыток
    name = netbox_interface.name
    addrs = []
    filter_query = mgmt_address(netbox_interface) # адрес интерфейса управления
    addrs.append(filter_query)
    
    responses, no_responses = multi_ping(addrs, timeout=0.5, retry=2,ignore_lookup_errors=True)
    print("icmp ping...")
    if filter_query in list(responses.keys()):
        print('{} is available...'.format(addrs))
        
        nr = create_nornir_session()
        sw = nr.filter(hostname = filter_query) # производим отбор по конкретному хосту
        get_int = sw.run(task=napalm_get, getters=['get_interfaces']) # получаем все интерфейсы с устройства в виде словаря
        
        for _ in range(attempts): # попытка подключения к устройству
            print('Attempting to connect {}...'.format(_+1))
    
            if get_int.failed == True: # попытка не удалась
                fail_count += 1
                time.sleep(attempt_timeout)
            
            else: # есть подключение
                print('Connection state is connected...')
        
                for device in get_int.values():
                    
                    interfaces = device.result['get_interfaces'].keys() # получаем интерфейсы как ключи словаря  
                    if name in (intf for intf in list(interfaces)):
                        print("Find {} for device {}...".format(name, device.host))              
                
                result = sw.run(netmiko_send_config,name="Configuration interface.../",config_commands=content)
                #result = sw.run(netmiko_send_config,name="Configuration interface.../",config_commands=cisco_config_interface(netbox_interface,event).split('\n'))
                
                #print_result(result)
                # > добавляем запись в журнал
                comment,level = 'All operations are performed','success'                
                journal_template_fill(comment,level)
                # <
                break
        if fail_count>=attempts:
             # > добавляем запись в журнал
            comment,level = 'Connection state is failed','error'                
            journal_template_fill(comment,level)
            # <
        sw.close_connections()
        print("Connection state is closed.")
    
    elif filter_query in no_responses:
        # > добавляем запись в журнал
        comment,level = '{} is not available'.format(addrs),'warning'                
        journal_template_fill(comment,level)
        # <
    
#push_config_interface(netbox_api.dcim.interfaces.get(136))


Удаляем (сбрасываем) конфигурацию интерфейса и добавляем дефолтные настройки

In [ ]:
def delete_config_intf(netbox_interface):
    """ 
    Удаляется соединение (cable) в Netbox, настройки порта коммутатора оставляем неизменными 
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :return: None
    """
    event='delete'
    try:
        content = cisco_config_interface(netbox_interface,event).split('\n')
        print("Delete interface {} config...".format(netbox_interface))
        push_config_interface(netbox_interface,content,event)
    except: print('No data...')

Создаем новую конфигурацию интерфейса

In [ ]:
def create_config_intf(netbox_interface):
    """ 
    Новый интерфейс
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :return: None
    """
    event = 'create'
    try:
        content = cisco_config_interface(netbox_interface,event).split('\n')
        print("Push new interface {} config...".format(netbox_interface))
        push_config_interface(netbox_interface,content,event)
    except: print('No data...')

Вносим изменения в конфигурацию интерфейса

In [ ]:
def update_config_intf(netbox_interface):
    """ 
    Обновление интерфейса
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :return: None
    """
    interface_name = netbox_interface.name
    event='update'
    try:
        content = cisco_config_interface(netbox_interface,event).split('\n')
        print("Updating interface {} config...".format(netbox_interface))
        push_config_interface(netbox_interface,content,event)
    except:
            # > добавляем запись в журнал
            comment,level = 'No data for {} {}'.format(get_event.lower(),interface_name),'informational'
            journal_template_fill(comment,level)
            # <

Отключаем интерфейс

In [ ]:
def shudown_intf(netbox_interface):
    """ 
    Отключение интерфейса
    :param netbox_interface: ссылка на объект интерфейса pynetbox
    :return: None
    """
    event='shutdown'
    try:
        content = cisco_config_interface(netbox_interface,event).split('\n')
        print("Updating interface {} config...".format(netbox_interface))
        push_config_interface(netbox_interface,content,event)
    except: print('No data...')

Управляем соединением

In [ ]:
def mng_cable():
    """  
    Соединение должно быть point-to-point между интерфейсами (Interface)
    Проверка полученных устройств на соответствие списку (access switch и user device)
    :return: Response(status=204)
    """
        
    devices_keys = ['role','device_id','intf_id'] # список ключей для словаря devices
    devices = []
    devices_names = []
    templates_roles = ['access_switch', 'user_device'] # получаем из netbox (произвольные данные)
    device_roles = []
    regex = "[a|b]_terminations"
    get_cable = request.json['data']
    get_event = request.json["event"]
    prechange = request.json['snapshots']['prechange']
    postchange = request.json['snapshots']['postchange']
    
    if get_event != "deleted": 
        #print("{} {}...".format(get_event.upper(),get_cable))
        #get_cable_id = conversion(list(netbox_api.dcim.cables.get('313')))
        
        for key in get_cable.keys(): # заполняем список device_value и объединяем с device_keys в словарь

            if re.match(regex, key) and len(get_cable[key])==1:  # отбираем нужные ключи из словаря по регулярке
                                                                    # при условии, что интерфейс один на устройство
                for _ in range(len(get_cable[key])):
                    device_id = get_cable[key][_]['object']['device']['id'] # ID устройства из json
                    devices_values = []
                    devices_names.append(netbox_api.dcim.devices.get(device_id).name)
                    devices_values.append(netbox_api.dcim.devices.get(device_id).device_role.slug) # роль устройства
                    devices_values.append(device_id) # id устройства
                    devices_values.append(get_cable[key][_]['object']['id']) # ID интерфейса устройства из json
                    devices.append(dict(zip(devices_keys,devices_values))) # получаем список из словарей
        
        for device in devices: # заполняем список ролей
            device_roles.append(device['role'])
        
        print("{} cable ID #{} between {}...".format(get_event.upper(),get_cable['id'], devices_names))
        
        if set(device_roles) == set(templates_roles): # проверяем, что получили устройства с разными ролями и в соответствии со списком    
            
            for device in devices:
                
                if device['role'] == templates_roles[0]: # нам нужен access switch
                    device_intf_id = device['intf_id'] # получаем ID интерфейса access switchа из нами созданного словаря            
            
            get_device_interface = netbox_api.dcim.interfaces.get(device_intf_id) # по ID находим интерфейс в netbox
            interface_name = get_device_interface.name
            print("Connection between {} and {}, switch access interface ID: {}...".format(device_roles[0],device_roles[1], device_intf_id))
            
            if get_device_interface.mgmt_only: # проверяем, является ли интерфейс management интерфейсов
                # > добавляем запись в журнал
                comment,level = '{} is management interface, no changes will be performed'.format(interface_name),'notification'                
                journal_template_fill(comment,level)
                # <
            
            else:
                
                if get_device_interface.enabled == False:
                    print('Interface {} was turned off before'.format(interface_name)) 
                
                elif get_event == "created": # Конфиг интерфейса будет добавлен

                    create_config_intf(netbox_interface=get_device_interface)

                elif get_event == "updated" and compare(prechange,postchange) != None: # Конфиг интерфейса будет изменен

                    update_config_intf(netbox_interface=get_device_interface) 
                
                else:
                    # > добавляем запись в журнал
                    comment,level = 'No data for {} {}'.format(get_event.lower(),interface_name),'informational'
                    journal_template_fill(comment,level)
                    # <

                
        else:
            # > добавляем запись в журнал
            comment,level = 'Devices must match the list of {}'.format(templates_roles),'notification'                
            journal_template_fill(comment,level)
            # <           
    else:
        # > добавляем запись в журнал
        comment,level = 'The cable {} is removed from the netbox, the interface settings are not touched'.format(get_cable_id['id'],'notification')
    
    return Response(status=204)

Управление интерфейсом

In [ ]:
def mng_int():
    """  
    Обновляем конфиг интерфейса
    :return: Response(status=204)
    """
        
    get_device_interface = netbox_api.dcim.interfaces.get(request.json['data']['id'])
    interface_name = get_device_interface.name
    get_event = request.json["event"]
    prechange = request.json['snapshots']['prechange']
    postchange = request.json['snapshots']['postchange']
    
    print("{} {}...".format(get_event.upper(), interface_name))
    
    if get_device_interface.mgmt_only: # проверяем, является ли интерфейс management интерфейсом       
        # > добавляем запись в журнал
        comment,level = '{} is management interface, no changes will be performed'.format(interface_name),'notification'                
        journal_template_fill(comment,level)
        # <
    
    else:
           
        if compare(prechange,postchange) == None:
            # > добавляем запись в журнал
            comment,level = 'No data for {} {}'.format(get_event.lower(),interface_name),'informational'
            journal_template_fill(comment,level)
            # <

        elif request.json['data']['enabled'] == False and prechange['enabled'] == True:
            # > добавляем запись в журнал
            comment,level = 'Interface {} is disabled on the device'.format(interface_name),'notification'              
            journal_template_fill(comment,level)
            # <
            shudown_intf(get_device_interface)
            
        elif request.json['data']['enabled'] == False and prechange['enabled'] == False:
            print('Interface {} was turned off before'.format(interface_name))              
                        
        else: 
            update_config_intf(get_device_interface)
        
    return Response(status=204)

In [ ]:
# Create a Flask instance
from flask import Flask
from nb_ipam_api import manage_interface_ip_address

# Create a Flask instance
app = Flask(__name__)

app.add_url_rule("/api/fixed_ip",
                 methods=["POST"],
                 view_func=manage_interface_ip_address)

app.add_url_rule("/api/cable_change",
                methods=['POST'],
                view_func=mng_cable)

app.add_url_rule("/api/int_update",
                 methods=['POST'],
                 view_func=mng_int)
    
if __name__ == "__main__": 
    app.run(host='0.0.0.0', port=8080)